<a href="https://colab.research.google.com/github/JiYoon03/AI-Modeling-Researcher_code/blob/main/Q1_A_JiYoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KnowledgeGraph 클래스:

회사별로 보험 상품 정보를 관리하는 지식 그래프입니다. 예시 데이터로 A 보험회사와 B 보험회사의 보험 상품을 정의하고 있습니다.

KnowledgeModule 클래스:

사용자의 질문에 대해, 지식 그래프와 연계하여 정확한 정보를 제공합니다. get_response 메서드는 회사명과 보험 종류를 기반으로 필요한 정보를 지식 그래프에서 찾아 반환합니다.

SearchModule 클래스:

사용자가 질문할 때마다, 공식 웹사이트와 같은 인증된 출처만을 검색하고 이를 반환합니다. 출처를 필터링하여 Hallucination을 방지하는 역할을 합니다.

RAGChatbot 클래스:

검색 모듈을 사용하여 출처를 확인하고, 지식 모듈을 통해 정확한 정보를 제공합니다. 두 모듈을 결합하여, Hallucination을 최소화하고 신뢰할 수 있는 정보를 제공합니다.

* 환화생명과 한화손해보험 웹사이트에서 스크래핑 하려고 시도 했었으나 보안상의 문제로 인해 실행이 되지 않아서 A보험회사 B 보험회사라는 가상의 웹페이지에서 스크래핑하는 시뮬레이터로 코딩했습니다.




In [11]:
class KnowledgeGraph:
    """
    지식 그래프 클래스는 특정 보험 상품과 관련된 정보를 저장하고 관리합니다.
    """
    def __init__(self):
        # example 데이터: 보험 회사와 상품 간의 관계
        self.graph = {
            "A 보험회사": ["건강보험", "자동차보험", "생명보험"],
            "B 보험회사": ["건강보험", "종합보험"]
        }

    def get_insurance_info(self, company, insurance_type):
        """
        회사와 보험 종류에 따른 정보를 반환합니다.
        """
        if company in self.graph and insurance_type in self.graph[company]:
            return f"{company}의 {insurance_type} 상품에 대한 정보"
        else:
            return f"{company}의 {insurance_type} 상품은 확인할 수 없습니다."


class KnowledgeModule:
    """
    지식 제공 모듈은 사용자의 질문에 대한 정확한 정보를 제공합니다.
    """
    def __init__(self, knowledge_graph):
        self.knowledge_graph = knowledge_graph
        self.knowledge_responses = {
            "보험 상품": "회사의 다양한 보험 상품을 확인해보세요. 예를 들어, 생명보험, 자동차보험, 건강보험 등이 있습니다.",
            "가입 절차": "가입 절차는 온라인 신청 후 서류 제출 및 상담을 진행합니다."
        }

    def get_response(self, query, company, insurance_type):
        """
        지식 기반의 응답을 제공합니다.
        - 주어진 회사와 보험 종류에 맞는 정보를 반환합니다.
        """
        if company and insurance_type:
            return self.knowledge_graph.get_insurance_info(company, insurance_type)
        # 기본적으로 질문에 대한 일반적인 정보 제공
        for key, response in self.knowledge_responses.items():
            if key in query:
                return response
        return "저는 해당 정보를 잘 모르겠어요. 다른 질문을 해주실래요?"


class SearchModule:
    """
    검색 모듈은 사용자의 질문에 대해 검색된 자료의 출처를 필터링하여 반환합니다.
    """
    def __init__(self):
        # 인증된 공식 웹사이트 URL을 가지고 있는 데이터베이스
        self.valid_sources = {
            "A 보험회사": "https://www.ainsurance.com",
            "B 보험회사": "https://www.binsurance.com"
        }

    def search(self, query, company):
        """
        사용자가 입력한 질문과 회사명에 기반하여 공식 출처를 검색하고 반환합니다.
        """
        if company in self.valid_sources:
            # 출처 필터링 후 공식 웹사이트만 반환
            return self.valid_sources[company]
        return "출처를 찾을 수 없습니다."


class RAGChatbot:
    """
    RAG 기반의 챗봇 시스템
    - 검색 모듈과 지식 그래프 활용하여 Hallucination을 줄임.
    """
    def __init__(self):
        self.knowledge_graph = KnowledgeGraph()
        self.knowledge_module = KnowledgeModule(self.knowledge_graph)
        self.search_module = SearchModule()

    def get_response(self, query, company=None, insurance_type=None):
        """
        사용자의 입력에 따라 적절한 응답을 생성
        """
        # 검색 모듈을 통해 관련된 출처를 검색
        search_result = self.search_module.search(query, company)

        # 지식 모듈을 통해 답변을 생성
        knowledge_response = self.knowledge_module.get_response(query, company, insurance_type)

        # Hallucination 방지를 위해 출처와 지식 모듈의 결과를 함께 반환
        return f"출처: {search_result}\n정보: {knowledge_response}"


# 2. 예시 대화 및 테스트 코드
if __name__ == "__main__":
    chatbot = RAGChatbot()

    # 테스트 케이스 1: A 보험회사의 건강보험 정보 요청
    print("사용자: A 회사의 건강보험에 대해 알려줘")
    print(chatbot.get_response("보험 상품", company="A 보험회사", insurance_type="건강보험"))

    # 테스트 케이스 2: B 보험회사의 자동차보험 정보 요청
    print("\n사용자: B 회사의 자동차보험에 대해 알려줘")
    print(chatbot.get_response("보험 상품", company="B 보험회사", insurance_type="자동차보험"))

    # 테스트 케이스 3: 정보가 없는 보험회사에 대한 요청
    print("\n사용자: C 회사의 건강보험에 대해 알려줘")
    print(chatbot.get_response("보험 상품", company="C 보험회사", insurance_type="건강보험"))

    # 테스트 케이스 4: 보험 관련 일반적인 정보 요청
    print("\n사용자: 가입 절차에 대해 알려줘")
    print(chatbot.get_response("가입 절차", company=None, insurance_type=None))

사용자: A 회사의 건강보험에 대해 알려줘
출처: https://www.ainsurance.com
정보: A 보험회사의 건강보험 상품에 대한 정보

사용자: B 회사의 자동차보험에 대해 알려줘
출처: https://www.binsurance.com
정보: B 보험회사의 자동차보험 상품은 확인할 수 없습니다.

사용자: C 회사의 건강보험에 대해 알려줘
출처: 출처를 찾을 수 없습니다.
정보: C 보험회사의 건강보험 상품은 확인할 수 없습니다.

사용자: 가입 절차에 대해 알려줘
출처: 출처를 찾을 수 없습니다.
정보: 가입 절차는 온라인 신청 후 서류 제출 및 상담을 진행합니다.
